In [1]:
from AOC_utils import get_day
import numpy as np

day = 5
input_data = get_day(day)

# print out first few lines to get a feel for the data
input_data[:7]

Day 5 input already downloaded


['seeds: 5844012 110899473 1132285750 58870036 986162929 109080640 3089574276 100113624 2693179996 275745330 2090752257 201704169 502075018 396653347 1540050181 277513792 1921754120 26668991 3836386950 66795009',
 '',
 'seed-to-soil map:',
 '3547471595 1239929038 174680800',
 '3052451552 758183681 481745357',
 '0 1427884524 1775655006',
 '2844087171 549819300 208364381']

In [2]:
example = '''
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
'''.split('\n')[1:-1]

In [25]:
def parse_data(data):
    # get seeds from first line of data
    seeds = np.array([int(x) for x in data[0].split(" ")[1:]])

    # for part 2, seeds work in pairs
    seed_pairs = seeds.reshape((-1 , 2))

    # rest of the input is used to construct mappings
    maps = {}
    ranges = []
    new_map = ""
    for i in range(1, len(data)):
        if "map" in data[i]:
            if len(ranges) > 0:
                maps[new_map] = ranges
                ranges = []
            new_map = data[i].split(" ")[0]
        else:
            new_range = [int(x) for x in data[i].split(" ") if x.isdigit()]
            if len(new_range) > 0:
                ranges.append(new_range)
        if i == len(data)-1:
            maps[new_map] = ranges

    def convert_ranges(sources, mapping):
        sources_copy = sources.copy()
        for rang in mapping:
            
            diff = rang[0] - rang[1]
            
            mapping_overlap = np.bitwise_and((sources >= rang[1]), (sources < rang[1]+rang[2]))
            sources_copy[mapping_overlap] = sources_copy[mapping_overlap] + diff

        return sources_copy

    def get_locations(seedsx):
        soils = convert_ranges(seedsx, maps["seed-to-soil"])
        fertilizers = convert_ranges(soils, maps["soil-to-fertilizer"])
        waters = convert_ranges(fertilizers, maps["fertilizer-to-water"])
        lights = convert_ranges(waters, maps["water-to-light"])
        temperatures = convert_ranges(lights, maps["light-to-temperature"])
        humidities = convert_ranges(temperatures, maps["temperature-to-humidity"])
        locations = convert_ranges(humidities, maps["humidity-to-location"])
        return locations
    
    locations = get_locations(seeds)
    print("part 1:", min(locations))
    

    # part 2 (brute force ~ 12 min)
    # min_locations = []
    # for i in range(len(seed_pairs)):
    #     pair = seed_pairs[i]
    #     seedsx = np.arange(pair[0]-1, pair[0]+pair[1]-1)
        
    #     locations = get_locations(seedsx)
    #     min_locations.append(min(locations))

    # print("part 2:", min(min_locations))


    # part 2: start with locations and work backwards (still brute force but faster ~ 3 min)
    def batch_invert_map(idxs, mapping):
        new_idxs = idxs.copy()
        for rang in mapping:
            diff = rang[1] - rang[0]
            new_idxs = np.where(np.bitwise_and((idxs >= rang[0]), (idxs < rang[0]+rang[2])), idxs + diff, new_idxs)
        return new_idxs

    break_flag = False
    stepsize = 1000000
    i = 0
    while True:
        irang = np.arange(i, i+stepsize)
        # start with the locations and work backwards
        x = batch_invert_map(irang, maps["humidity-to-location"])
        x = batch_invert_map(x, maps["temperature-to-humidity"])
        x = batch_invert_map(x, maps["light-to-temperature"])
        x = batch_invert_map(x, maps["water-to-light"])
        x = batch_invert_map(x, maps["fertilizer-to-water"])
        x = batch_invert_map(x, maps["soil-to-fertilizer"])
        x = batch_invert_map(x, maps["seed-to-soil"])

        # check whether any of the locations are in the seeds
        for pair in seed_pairs:
            seed_matches = np.bitwise_and(x >= pair[0], x < pair[0]+pair[1])
            if np.any(seed_matches):
                print("part 2:", min(irang[seed_matches]))
                break_flag = True
                break
        if break_flag:
            break
        i += stepsize

    # part 2: but for real this time
    def collpase_maps(map0, map1):
        # for each range in map0 loop through each range in map1 and see if they overlap
        # if they do, then collapse the range by 
        

    # pretty sure the 'correct' way to solve this problem is by collapsing the mappings into a single mapping
    # and then the same code used to collpase the mappings, could be used to collapse the range of seeds
    # additionally I think you can filter out bad mappings that wouldn't result in a low location

In [27]:
parse_data(input_data)

part 1: 825516882
part 2: 136096660


In [26]:
parse_data(example)

part 1: 35
part 2: 46
